Static Task Assignment

In [1]:
%%sh
cat > a.c << EOF
#include <stdio.h>
#include <time.h>
#include <mpi.h>

#define WIDTH 1000
#define HEIGHT 1000
#define MAX_ITER 100

struct complex{
  double real;
  double imag;
};

int cal_pixel(struct complex c) {


            double z_real = 0;
            double z_imag = 0;

            double z_real2, z_imag2, lengthsq;

            int iter = 0;
            do {
                z_real2 = z_real * z_real;
                z_imag2 = z_imag * z_imag;

                z_imag = 2 * z_real * z_imag + c.imag;
                z_real = z_real2 - z_imag2 + c.real;
                lengthsq =  z_real2 + z_imag2;
                iter++;
            }
            while ((iter < MAX_ITER) && (lengthsq < 4.0));

            return iter;

}

void save_pgm(const char *filename, int image[HEIGHT][WIDTH]) {
    FILE* pgmimg;
    int temp;
    pgmimg = fopen(filename, "wb");
    fprintf(pgmimg, "P2\n");
    fprintf(pgmimg, "%d %d\n", WIDTH, HEIGHT);
    fprintf(pgmimg, "255\n");
    int count = 0;

    for (int i = 0; i < HEIGHT; i++) {
        for (int j = 0; j < WIDTH; j++) {
            temp = image[i][j];
            fprintf(pgmimg, "%d ", temp);
        }
        fprintf(pgmimg, "\n");
    }
    fclose(pgmimg);
}


int main() {

    int image[HEIGHT][WIDTH];
    double AVG = 0;
    int N = 10;
    double total_time[N];
    struct complex c;

        MPI_Init(NULL,NULL);
        int world_size;
        MPI_Comm_size(MPI_COMM_WORLD, &world_size);
        int rank;
        MPI_Comm_rank(MPI_COMM_WORLD, &rank);
        int rows_per_proc = HEIGHT/world_size;
        int start_row = rank * rows_per_proc;
        int end_row = start_row + rows_per_proc;
        if (rank == world_size - 1)
            end_row = HEIGHT;

    for (int k=0; k<N; k++){
        clock_t start_time = clock();
        int i, j;

            int count=0;
            for (i = start_row; i < end_row; i++) {
                for (j = 0; j < WIDTH; j++) {
                    count=count+1;
                    c.real = (j - WIDTH / 2.0) * 4.0 / WIDTH;
                    c.imag = (i - HEIGHT / 2.0) * 4.0 / HEIGHT;
                    image[i][j]=cal_pixel(c);
                    //printf("sent %d and %d start=%d end=%d",rank,count,start_row,end_row);
                }
            }

        MPI_Gather(image[start_row], rows_per_proc * WIDTH, MPI_INT, &image[start_row], rows_per_proc * WIDTH, MPI_INT, 0, MPI_COMM_WORLD);

        clock_t end_time = clock();

        total_time[k] = ((double)(end_time - start_time)) / CLOCKS_PER_SEC;
        printf("Execution time of trial [%d]: %f seconds\n", i , total_time[k]);
        AVG += total_time[k];
    }

    MPI_Finalize();

    if(rank==0){
    save_pgm("mandelbrot.pgm", image);
    printf("The average execution time of 10 trials is: %f ms\n", AVG/N*1000);
    }
    return 0;
}

EOF
ls -l

total 8
-rw-r--r-- 1 root root 2829 Feb 18 20:12 a.c
drwxr-xr-x 1 root root 4096 Feb 14 14:28 sample_data


In [2]:
!mpicc a.c

In [3]:
!mpirun --oversubscribe -np 10 --allow-run-as-root a.out

Execution time of trial [200]: 0.004666 seconds
Execution time of trial [1000]: 0.003485 seconds
Execution time of trial [300]: 0.021169 seconds
Execution time of trial [400]: 0.017663 seconds
Execution time of trial [800]: 0.018055 seconds
Execution time of trial [400]: 0.017665 seconds
Execution time of trial [700]: 0.036934 seconds
Execution time of trial [600]: 0.038835 seconds
Execution time of trial [100]: 0.038358 seconds
Execution time of trial [900]: 0.051514 seconds
Execution time of trial [500]: 0.038680 seconds
Execution time of trial [200]: 0.037317 seconds
Execution time of trial [1000]: 0.036790 seconds
Execution time of trial [300]: 0.023011 seconds
Execution time of trial [400]: 0.018099 seconds
Execution time of trial [800]: 0.039032 seconds
Execution time of trial [600]: 0.036687 seconds
Execution time of trial [700]: 0.037418 seconds
Execution time of trial [100]: 0.038623 seconds
Execution time of trial [900]: 0.038488 seconds
Execution time of trial [500]: 0.03676

Dynamic Task Assignment

In [10]:
%%sh
cat > a.c << EOF
#include <stdio.h>
#include <mpi.h>
#include <time.h>

#define WIDTH 1000
#define HEIGHT 1000
#define MAX_ITER 100

struct complex{
  double real;
  double imag;
};

int cal_pixel(struct complex c) {
    double z_real = 0;
    double z_imag = 0;
    double z_real2, z_imag2, lengthsq;
    int iter = 0;
    do {
        z_real2 = z_real * z_real;
        z_imag2 = z_imag * z_imag;
        z_imag = 2 * z_real * z_imag + c.imag;
        z_real = z_real2 - z_imag2 + c.real;
        lengthsq =  z_real2 + z_imag2;
        iter++;
    } while ((iter < MAX_ITER) && (lengthsq < 4.0));
    return iter;
}

void save_pgm(const char *filename, int image[HEIGHT][WIDTH]) {
    FILE* pgmimg;
    int temp;
    pgmimg = fopen(filename, "wb");
    fprintf(pgmimg, "P2\n");
    fprintf(pgmimg, "%d %d\n", WIDTH, HEIGHT);
    fprintf(pgmimg, "255\n");
    int count = 0;
    for (int i = 0; i < HEIGHT; i++) {
        for (int j = 0; j < WIDTH; j++) {
            temp = image[i][j];
            fprintf(pgmimg, "%d ", temp);
        }
        fprintf(pgmimg, "\n");
    }
    fclose(pgmimg);
}

int main(int argc, char *argv[]) {
    int image[HEIGHT][WIDTH];
    struct complex c;
    double AVG = 0;
    int N = 10;
    double total_time[N];
    int rank, size;

    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    int rows_per_process = HEIGHT / size;
    int start_row = rank * rows_per_process;
    int end_row = start_row + rows_per_process;

    for (int k = 0; k < N; k++) {
        clock_t start_time = clock();
        for (int i = start_row; i < end_row; i++) {
            for (int j = 0; j < WIDTH; j++) {
                c.real = (j - WIDTH / 2.0) * 4.0 / WIDTH;
                c.imag = (i - HEIGHT / 2.0) * 4.0 / HEIGHT;
                image[i][j] = cal_pixel(c);
            }
        }
        MPI_Barrier(MPI_COMM_WORLD);

        // Dynamic task assignment:
        if (rank != 0) {
            MPI_Send(&start_row, 1, MPI_INT, 0, 0, MPI_COMM_WORLD);
            MPI_Send(&image[start_row][0], rows_per_process * WIDTH, MPI_INT, 0, 1, MPI_COMM_WORLD);
        } else {
            for (int p = 1; p < size; p++) {
                MPI_Recv(&start_row, 1, MPI_INT, p, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
                MPI_Recv(&image[start_row][0], rows_per_process * WIDTH, MPI_INT, p, 1, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
            }
        }
        clock_t end_time = clock();
        total_time[k] = ((double)(end_time - start_time)) / CLOCKS_PER_SEC;
        printf("Execution time of trial [%d]: %f seconds\n", k, total_time[k]);
        AVG += total_time[k];
    }

    MPI_Finalize();
    if(rank==0){
        save_pgm("mandelbrot.pgm", image);
    printf("The average execution time of 10 trials is: %f ms", AVG / N * 900);
    }
    return 0;
}
EOF
ls -l

total 2216
-rw-r--r-- 1 root root    2839 Feb 18 20:15 a.c
-rwxr-xr-x 1 root root   16640 Feb 18 20:15 a.out
-rw-r--r-- 1 root root    2839 Feb 18 20:15 b.c
-rw-r--r-- 1 root root 2235243 Feb 18 20:13 mandelbrot.pgm
drwxr-xr-x 1 root root    4096 Feb 14 14:28 sample_data


In [11]:
!mpicc a.c

In [12]:
!mpirun --oversubscribe -np 10 --allow-run-as-root a.out

Execution time of trial [0]: 0.033348 seconds
Execution time of trial [0]: 0.047664 seconds
Execution time of trial [0]: 0.046972 seconds
Execution time of trial [0]: 0.035516 seconds
Execution time of trial [0]: 0.034368 seconds
Execution time of trial [0]: 0.034540 seconds
Execution time of trial [0]: 0.030532 seconds
Execution time of trial [0]: 0.045007 seconds
Execution time of trial [0]: 0.035675 seconds
Execution time of trial [0]: 0.037525 seconds
Execution time of trial [1]: 0.035117 seconds
Execution time of trial [1]: 0.036061 seconds
Execution time of trial [1]: 0.034334 seconds
Execution time of trial [1]: 0.035036 seconds
Execution time of trial [1]: 0.035024 seconds
Execution time of trial [1]: 0.035024 seconds
Execution time of trial [1]: 0.035483 seconds
Execution time of trial [1]: 0.035482 seconds
Execution time of trial [1]: 0.035507 seconds
Execution time of trial [1]: 0.035019 seconds
Execution time of trial [2]: 0.035063 seconds
Execution time of trial [2]: 0.035